## IMPORT PACKAGES

In [1]:
import threading
from time import sleep as sp
import random

## INITIALIZATION

In [2]:
custReady = threading.Semaphore(0)   #for synchronization
barberReady= threading.Semaphore(0)  #for synchronization
access_chair= threading.Semaphore(1) # for mutual exclusion, as one customer should access one chair at a time 
global chairs

sleep=0                     # for barber is awake, sleep = 1 means barber is sleeping
total=0                     # total no. of customer including one which getting hair cut
waiting_list=[]             # name of waiting cusomer sitting on waiting chair
cust_getting_hair_cut=""    # name of customer currently getting hair cut
cust_threads=[]             #for name of all customers

## FUNCTIONS CALLED IN BARBER AND CUSTOMER

In [3]:
def get_hair_cut(name): # called for hair cut
    print(f"Customer {name} has called getHairCut()\n")
    return
def cut_hair():  # cutting of hair
    print(f"Barber has called cutHair()\nBarber: Cutting hair of {cust_getting_hair_cut}\n")
    return
def balk(name):  # leave the shop
    print(f"Customer: {name} is trying to enter waiting room\nCustomer : shop full, balk() called for {name}\n")


## CUSTOMER AND BARBER

In [4]:
def barber():
    global total, chairs, sleep , waiting_list, cust_getting_hair_cut
    while True:
        if (len(waiting_list)==0 and total==0): #if no customer in the shop barber goes to sleep
            print("\nThere are no customers in the barber shop\n\n\t\t Barber: No customers in the shop! , I am going to sleep  zzz....\n")
            sleep=1
        custReady.acquire() # semwait (waiting for customer)
        
        if (len(waiting_list)>0): # if there is someone on waiting chair
            print("Waiting customers are mentioned below :\n",end='')
            for i in waiting_list:
                print(i)
            print()        
        access_chair.acquire()# semwait (mutual exclusion)
        
        if(len(waiting_list)>0):  # if customer waiting 
            chairs+=1
        access_chair.release()    # semsignal (mutual exclusion)
        barberReady.release()  # semsignal (barber can start working)
        
        sp(2)      # wait for 2 sec
        cut_hair() # barber cut hair
        sp(4)      # wait for 4 sec
        print(f"Barber: Done with hair cutting of {cust_getting_hair_cut}\n")
        if(len(waiting_list)>0 and  cust_getting_hair_cut in waiting_list):
            waiting_list.remove(cust_getting_hair_cut)  # remove customer from waiting list who are done with hair cut
        cust_getting_hair_cut = ""  
        total -=1   # customer after done with haircut leaves the shop

def customer(name):
    global total,chairs, sleep , waiting_list, cust_getting_hair_cut , chairs
    
    access_chair.acquire() # semwait (mutual exclusion)
    if (chairs>0 or total==0):
        total+=1
        if (chairs> 0 or (total==0 and chairs!=0)): # if chairs available or no customer in shop
            chairs-=1
            waiting_list.append(name)
            if (total==0):
                print(f"Customer: {name} has entered in barber shop \n")
            else:
                print(f"Customer: {name} has entered waiting room.\n\nNow total chairs available in waiting room are : ", chairs,"\n")
        elif (total ==0 and chairs==0): # if there is no waiting chair
            print(f"Customer: {name} has entered in barber shop \n")
        access_chair.release()      # semsignal (mutual exclusion)
        custReady.release()  # semsignal (customers are present in the barber shop)
        barberReady.acquire()    # semwait (waiting for barber)
        if (sleep==1): # barber is sleeping
            print(f"Customer : {name} is waking up the barber \n")
            #print(f"Now the waiting room chairs are:{chairs}\n")
            sleep=0 #customer wakes up the barber
        cust_getting_hair_cut=name
        get_hair_cut(name)
        if chairs != 0:
            print(f"Now the waiting room chairs are {chairs}\n")
            
    else:           # no chair available means the shop is full
        access_chair.release() # semsignal (mutual exclusion)
        balk(name)   #customer leaves the shop




## MAIN INTERFACE

In [5]:
print("\t\t=======WELCOME TO BARBER SHOP==========\n")

barber_thread= threading.Thread(name="Barber" , target=barber) # making barber thread
barber_thread.start()

while True:
    if (len(cust_threads)):
        for t in cust_threads:     # Wait for all of cust_threads to finish
            t.join() 
    
    if (len(waiting_list)==0):
        custs=[]
        sp(1)
        exit=input("Do you want to continue? ")
        if (exit!='y' and exit!='Y'):
            break
        print('\nThe barber room contains only one barber chair.\n')
        chairs= int(input("How many waiting chairs do you want in the barber shop ? "))        
        cust_no= int(input("How many customers do you want in your scenerio ? "))
        if (cust_no==0):   # if there is no customer in the shop, no one will wake the barber up
            print("\nThere are no customers in the barber shop\n\n\t\t Barber: No customers in the shop! , I am going to sleep  zzz....\n")        
        for i in range (cust_no):
          while True:
            cust_name=input("Enter Customer "+str(i+1)+" Name: ")
            if (cust_name.isdigit() or cust_name in ['','*','@','%','$','^','!','#','&','/']):  #checking if the name is entered properly
                print('Enter proper name!')
            else:
                custs.append(cust_name.capitalize())
                break
        print('')
        for index,cust in enumerate(custs[:cust_no]):
            sp(1)
            customer_thread=threading.Thread(name="Customer", target=customer, args=[f'{cust}']) # making customers threads
            customer_thread.start()
            cust_threads.append(customer_thread)

		=======WELCOME TO BARBER SHOP==========


There are no customers in the barber shop

		 Barber: No customers in the shop! , I am going to sleep  zzz....

Do you want to continue? Y

The barber room contains only one barber chair.

How many waiting chairs do you want in the barber shop ? 2
How many customers do you want in your scenerio ? 2
Enter Customer 1 Name: Bisma
Enter Customer 2 Name: Ayesha

Customer: Bisma has entered waiting room.

Now total chairs available in waiting room are :  1 

Waiting customers are mentioned below :
Bisma

Customer : Bisma is waking up the barber 

Customer Bisma has called getHairCut()

Now the waiting room chairs are 2

Customer: Ayesha has entered waiting room.

Now total chairs available in waiting room are :  1 

Barber has called cutHair()
Barber: Cutting hair of Bisma

Barber: Done with hair cutting of Bisma

Waiting customers are mentioned below :
Ayesha

Customer Ayesha has called getHairCut()

Now the waiting room chairs are 2

Barber has c